## Measure contribution of Major merger, minor merger, and smooth accretion for only the 'safe' samples...?. 
Because tree bad link more likely occur at major merger events, I guess the 'safe' samples have less major mergers than the total sample. 

## import analysis.Major_Minor_accretion as mma


Largest halo in the refinement region is not the main cluster.
Is the largest NP halo the main cluster? 
To check it, color halos in NP.

No, max_np galaxy/halo is the main galaxy/halo.
But 'rvir' value is wrong.

and fixed.

In [1]:
import utils.sampling as smp
import matplotlib.pyplot as plt
import tree
import pickle
import tree.halomodule as hmo
import numpy as np
from analysis.misc import load_cat
import scipy.stats
import tree.ctutils as ctu
from analysis.evol_lambda import MainPrg
import draw
import load
import analysis.evol_lambda as evl
import analysis.Major_Minor_accretion as mma
import analysis.misc as amsc
import tree.ctutils as ctu
import utils.match as mtc
# Read a single galaxy evolution catalog.

from analysis.MajorMinorAccretion_module import *
from analysis.all_plot_modules import *

verbose=True
# In[4]:
base = './'
cdir = ['catalog/', 'HM/', 'catalog_GM/', "easy_final/"][3]

clusters = ['01605', '07206', \
            '35663', '24954', '49096', \
            '05427', '05420', '29172', \
            '29176', '10002', '36415', \
            '06098', '39990', '36413', \
            '17891', '04466']
# parameters used for lambda_arr clipping.
#ind_upper = 20
#ind_lower = 20
#sig_upper = 2.0
#sig_lower = 2.0

 # 62: z = 1.666
nout_fi = 187

minimum_good_snap = 87


In [2]:
def body(clusters,
         dist_gal_scale_in=5,
         dist_gal_scale_out=10,
         dt_before=0.5,
         dt_after=0.5,
         dt_settle=0.5,
         load=False,
         nout_ini = 37,
         filter_small=True,
         min_mass_ratio = 0.05,
         measure_delta_savefig=False,
         find_merger_epoch_plot=False,
         cdir=""):

    suffix = "_{}_{}_{}_{}_{}_{}_{}".format(dist_gal_scale_in,
    dist_gal_scale_out,dt_before,dt_after,dt_settle,nout_ini,
                                           min_mass_ratio)
    if filter_small: 
        suffix = suffix + "_filtered_"

    if load:
        return pickle.load(open("main_prgs_final_augmented" + suffix + ".pickle", 'rb'))
        
    else:
        mpgs = []
        for cluster in clusters:
            print(cluster)
            wdir = base + cluster + '/'

            # Serialize catalogs. -> Only main galaxies
            # main galaxy list
            alltrees = ctu.load_tree(wdir, is_gal=True)
            ad = alltrees.data
            tn = ad[ad['nout'] == nout_fi]

            cat = load_cat(wdir + cdir + 'catalog' + str(nout_fi) + '.pickle')
            #idx_all = [tn['id'][tn['Orig_halo_id'] == id_final][0] for id_final in cat['id']]
            idx_all = cat['idx'][cat["idx"] > 0].astype(int) # why idx are float???

            mpg_tmp = []
            for i, idx in enumerate(idx_all):
                #print(i, idx)

                mpg_tmp.append(MainPrg(ad, idx))
            #    mpg_tmp =[MainPrg(ad, idx) for idx in idx_all]
            for nout in range(nout_ini, nout_fi + 1):
                cat = load_cat(wdir + cdir + 'catalog' + str(nout) + '.pickle')
                for gal in mpg_tmp:
                    gal.set_data(cat, nout)
                    gal.cluster = int(cluster)
        #        print(nout)
            # get rid of galaxies with too short tree.
            mpg_tmp = [gg for gg in mpg_tmp if sum(gg.data["reff"] > 0) > minimum_good_snap]
            for gal in mpg_tmp:
                gal.fill_missing_data()
                gal.clip_non_detection()
                gal.smoothed_lambda_org = mma.smooth(gal.data["lambda_r"], window_len=15)[:-1]
                gal.smoothed_r = mma.smooth(gal.data["reff"], window_len=15)[:-1]
                gal.smoothed_lambda = mma.smooth(l_at_smoothed_r(gal, npix_per_reff=5), window_len=15)[:-1]

            # save for each cluser
            with open(wdir + "main_prgs" + suffix + ".pickle", "wb") as f:
                pickle.dump(mpg_tmp, f)    
                
            # Find_merger_epochs needs smoothed_r
            find_merger_epochs(alltrees,
                               idx_all,
                               mpg_tmp,
                               nout_ini=nout_ini,
                               dist_gal_scale_in=dist_gal_scale_in,
                               dist_gal_scale_out=dist_gal_scale_out,
                               min_mass_ratio = min_mass_ratio,
                               mass_ratio='early',
                               verbose=False,
                               do_plot=find_merger_epoch_plot,
                               max_rgal=40,
                               pdf_fname=str(cluster) + "merger_ratio_epoch" + suffix + ".pdf")
            

            while len(mpg_tmp) > 0:
                mpgs.append(mpg_tmp.pop())

        if filter_small:
            for gal in mpgs:
                # Keep only the largest merger among multiple mergers
                filter_small_mergers(gal.merger)

        with open("main_prgs" + suffix + ".pickle", 'wb') as f:
            mpgs.pop(2)
            pickle.dump(mpgs, f)

        measure_delta(mpgs,
                      dt_before=dt_before,
                      dt_after=dt_after,
                      dt_settle=dt_settle,
                      nout_ini=nout_ini,
                      savefig=measure_delta_savefig,
                      figname="figs/measure_delta" + suffix)

        Maj_min_acc_ratio(mpgs, major_ratio=4)

        with open("main_prgs_final_augmented" + suffix + ".pickle", 'wb') as f:
            pickle.dump(mpgs, f)
    return mpgs

In [6]:
# Skewed Gaussian fit Major/Minor merger distribution and 
# qunatify how efficient major/minor mergers are in changing rotation paramter.
def skew_fit(xvals, yvals, amp=10, center=0, sigma=1, gamma=0):
    from lmfit.models import SkewedGaussianModel
    model = SkewedGaussianModel()

    # set initial parameter values
    params = model.make_params(amplitude=amp, center=center, sigma=sigma, gamma=gamma)

    # adjust parameters  to best fit data.
    return model.fit(yvals, params, x=xvals)


def draw_kdes_fit(dlM, dlm, dlo, dlt, ax
                  , nevents
                  , lw=1.5
                  , excess=False
                  , do_fit=False):

        dM = kde_den(dlM)
        dm = kde_den(dlm)
        do = kde_den(dlo)
        dtot = kde_den(dlt)

        xs=np.linspace(-0.7,0.7,51)
        if excess:
            i_positive = np.linspace(0.0,0.6,26)
            dM_curve = dM(xs) - np.concatenate((dM(i_positive)[::-1], dM(i_positive[1:])))
            dm_curve = dm(xs) - np.concatenate((dm(i_positive)[::-1], dm(i_positive[1:])))
            do_curve = do(xs) - np.concatenate((do(i_positive)[::-1], do(i_positive[1:])))
            dtot_curve = dtot(xs) - np.concatenate((dtot(i_positive)[::-1], dtot(i_positive[1:])))
        else:
            dM_curve = dM(xs)
            dm_curve = dm(xs)
            do_curve = do(xs)
            dtot_curve = dtot(xs)

        nM = len(dlM)
        nm = len(dlm)
        no = len(dlo)
        ntot = len(dlt)


        Mlabel="Major \n" +r"$N_{g}(N_{e})$" + " = {}({})".format(nM, nevents[0])
        mlabel="Minor \n" +r"$N_{g}(N_{e})$" + " = {}({})".format(nm, nevents[1])
        olabel="Rest  \n" +r"$N_{g}$" + " = {}".format(no)
        totlabel="Total\n" +r"$N_{g}$" + " = {}".format(ntot)
        ax.plot(xs, dM_curve*nM/ntot, label=Mlabel, lw=lw, color="r")
        ax.plot(xs, dm_curve*nm/ntot, label=mlabel, lw=lw, color="g")
        ax.plot(xs, do_curve*no/ntot, label=olabel, lw=lw, color="b")
        ax.plot(xs, dtot_curve, label=totlabel, lw=lw, color="black")
        
        ax.set_ylim([0, 1.15*ax.get_ylim()[1]])
        
        if do_fit:
            fits=[]
            fits.append(skew_fit(xs, dM_curve*nM/ntot, amp=nM/ntot, center=0, sigma=0.3, gamma=0))
            fits.append(skew_fit(xs, dm_curve*nm/ntot, amp=nm/ntot, center=0, sigma=0.3, gamma=0))
            fits.append(skew_fit(xs, do_curve*no/ntot, amp=no/ntot, center=0, sigma=0.3, gamma=0))
            fits.append(skew_fit(xs, dtot_curve, amp=1, center=0, sigma=0.3, gamma=0))
            #print(dM_curve*nM/ntot, rr.best_fit)
            ax.plot(xs, fits[0].best_fit, lw=lw, color="r", linestyle=":")
            ax.plot(xs, fits[1].best_fit, lw=lw, color="g", linestyle=":")
            ax.plot(xs, fits[2].best_fit, lw=lw, color="b", linestyle=":")
            ax.plot(xs, fits[3].best_fit, lw=lw, color="black", linestyle=":")

            return fits



# This part is already done. 
mpgs = body(clusters,
        dist_gal_scale_in=5,
        dist_gal_scale_out=10,
        dt_before=0.5,
        dt_after=0.5,
        dt_settle=0.5,
        load=False,
        nout_ini=37,
        filter_small=True,
        min_mass_ratio = 0.01, # 0.1로 테스트
        measure_delta_savefig=False,
        find_merger_epoch_plot=False,
        cdir=cdir)# z = 3

In [3]:
wdir = '/home/hoseung/Work/data/'

In [4]:
suffix1 = "_5_10_0.5_0.5_0.5_"
#suffix2 = ["37_0.01", "37_0.1", "37_0.001", "27_0.01"][2]
suffix2 = "37_0.02"
mpgs = pickle.load(open(wdir + "all_prgs/main_prgs_final_augmented" + suffix1 + suffix2 + "_filtered_.pickle", "rb"))

In [5]:
def kde_sci(mpgs,
            mstar_cut_hard = 5e9,
            mcut = 3.3e10,
            hist=False,
            shade=False,
            kde=True,
            norm_hist=False,
            detected=True,
            maj_ratio=4,
            excess=False,
            fname=None,
            img_scale=1.5,
            do_fit=False):

    fontsize_ticks = 6 * img_scale
    fontsize_tick_label = 8 * img_scale
    fontsize_legend = 5 * img_scale

    from matplotlib.ticker import NullFormatter

    #examplegal = mpgs[449]
    examplegal = None

    l_dl_e = []
    l_mr_e = []
    l_mass_e = []

    s_dl_e = []
    s_mr_e = []
    s_mass_e = []

    l_dlt_g=[]
    l_dlo_g=[]
    l_dlM_g=[]
    l_dlm_g=[]
    l_mass_g=[]

    s_dlt_g=[]
    s_dlo_g=[]
    s_dlM_g=[]
    s_dlm_g=[]
    s_mass_g=[]

    M_changed = 0
    m_changed = 0
    no_merger_count = 0
    count = 0
    Maj_small = 0
    for i, gal in enumerate(mpgs):
        mgal = gal.data["mstar"][0]
        if mgal > mstar_cut_hard:
            delta_lambda_tot = np.average(gal.data['lambda_r'][:5]) - np.average(gal.data['lambda_r'][-5:])
            delta_lambda_major = 0
            delta_lambda_minor = 0

            # Large
            if mgal > mcut:
                if hasattr(gal, "merger"):
                    if gal.merger is not None:
                        l_dl_e.extend(gal.merger.delta_l)
                        l_mr_e.extend(gal.merger.mr)
                        for dl, mr in zip(gal.merger.delta_l, gal.merger.mr):
                            if (mr < maj_ratio) and (dl > -1):
                                delta_lambda_major = delta_lambda_major + dl
                            if (mr > maj_ratio) and (dl > -1):
                                delta_lambda_minor = delta_lambda_minor + dl

                delta_lambda_other = delta_lambda_tot - delta_lambda_major - delta_lambda_minor
                l_dlt_g.append(delta_lambda_tot)
                l_dlo_g.append(delta_lambda_other)
                l_dlM_g.append(delta_lambda_major)
                l_dlm_g.append(delta_lambda_minor)
            # small
            else:
                #s_mass_g.append(mgal)
                if hasattr(gal, "merger"):
                    if gal.merger is not None:
                        s_dl_e.extend(gal.merger.delta_l)
                        s_mr_e.extend(gal.merger.mr)
                        for dl, mr in zip(gal.merger.delta_l, gal.merger.mr):
                            if (mr < maj_ratio) and (dl > -1):
                                delta_lambda_major = delta_lambda_major + dl
                            if (mr > maj_ratio) and (dl > -1):
                                delta_lambda_minor = delta_lambda_minor + dl


                    delta_lambda_other = delta_lambda_tot - delta_lambda_major - delta_lambda_minor
                    s_dlt_g.append(delta_lambda_tot)
                    s_dlo_g.append(delta_lambda_other)
                    s_dlM_g.append(delta_lambda_major)
                    s_dlm_g.append(delta_lambda_minor)

    l_dlt_g = np.array(l_dlt_g)
    l_dlo_g = np.array(l_dlo_g)
    l_dlM_g = np.array(l_dlM_g)
    l_dlm_g = np.array(l_dlm_g)
    #l_mass_g = np.array(l_mass_g)

    s_dlt_g = np.array(s_dlt_g)
    s_dlo_g = np.array(s_dlo_g)
    s_dlM_g = np.array(s_dlM_g)
    s_dlm_g = np.array(s_dlm_g)
    #s_mass_g = np.array(s_mass_g)

    # detected
    l_dlM_g = l_dlM_g [l_dlM_g !=0]
    #l_dlM_M = l_mass_g[l_dlM_g !=0]
    l_dlm_g = l_dlm_g [l_dlm_g !=0]
    #l_dlm_M = l_mass_g[l_dlm_g !=0]
    #l_dlo_M = l_mass_g

    s_dlM_g = s_dlM_g [s_dlM_g !=0]
    #s_dlM_M = s_mass_g[s_dlM_g !=0]
    s_dlm_g = s_dlm_g [s_dlm_g !=0]
    #s_dlm_M = s_mass_g[s_dlm_g !=0]
    #s_dlo_M = s_mass_g


    l_dl_e = np.array(l_dl_e)
    l_mr_e = np.array(l_mr_e)
    #l_mass_e = []

    s_dl_e = np.array(s_dl_e)
    s_mr_e = np.array(s_mr_e)
    #s_mass_e = []

    fig, axs = plt.subplots(3, sharex=True)
    fig.set_size_inches(4.75,7)
    plt.subplots_adjust(hspace=0.01)

    all_dlM_g = np.concatenate((l_dlM_g,s_dlM_g))
    all_dlm_g = np.concatenate((l_dlm_g,s_dlm_g))
    all_dlo_g = np.concatenate((l_dlo_g,s_dlo_g))
    all_dlt_g = np.concatenate((l_dlt_g,s_dlt_g))

    skew_fits=[]

    skew_fits.append(
              draw_kdes_fit(all_dlM_g,
              all_dlm_g,
              all_dlo_g,
              all_dlt_g,
              axs[0],
              [sum(s_mr_e < maj_ratio) + sum(l_mr_e < maj_ratio),
               sum(s_mr_e > maj_ratio) + sum(l_mr_e > maj_ratio),
               len(all_dlo_g)],
              excess=excess,
              do_fit=do_fit))

    skew_fits.append(
              draw_kdes_fit(l_dlM_g,
              l_dlm_g,
              l_dlo_g,
              l_dlt_g,
              axs[1],
              [sum(l_mr_e < maj_ratio),
               sum(l_mr_e > maj_ratio),
               len(l_dlo_g)],
              excess=excess,
              do_fit=do_fit))

    skew_fits.append(
              draw_kdes_fit(s_dlM_g,
              s_dlm_g,
              s_dlo_g,
              s_dlt_g,
              axs[2],
              [sum(s_mr_e < maj_ratio),
               sum(s_mr_e > maj_ratio),
               len(s_dlo_g)],
              excess=excess,
              do_fit=do_fit))

    axs[0].set_xlim([-0.6,0.6])

    for ax in axs:
        ax.xaxis.grid()
        leg = ax.legend(fontsize=fontsize_legend)
        leg.get_frame().set_alpha(0.5)
        ax.yaxis.set_major_formatter(NullFormatter())
        ax.set_ylabel("relative probability", fontsize=fontsize_tick_label)

    axs[2].set_xlabel(r"$\Delta \lambda_{R_{eff}}$", fontsize=fontsize_tick_label, family="Liberation Sans")
    axs[2].tick_params(labelsize=fontsize_ticks)
    axs[2].set_xlim([-0.7,0.6])
    #axs[0].legend(fontsize=12)


    axs[0].text(0.05, 0.87, "(A)", weight="bold", transform=axs[0].transAxes, fontsize=fontsize_ticks)
    axs[0].text(0.15, 0.87, "All",transform=axs[0].transAxes, fontsize=fontsize_ticks)
    axs[1].text(0.05, 0.87, "(B) ", weight="bold",transform=axs[1].transAxes, fontsize=fontsize_ticks)
    axs[1].text(0.15, 0.87, r"$log_{10}M_{\star} > $ " +"{:.1f}".format(np.log10(mcut))
                , fontsize=fontsize_ticks
                , transform=axs[1].transAxes)
    axs[2].text(0.05, 0.87, "(C) ", weight="bold",transform=axs[2].transAxes, fontsize=fontsize_ticks)
    axs[2].text(0.15, 0.87, r"$log_{10}M_{\star} < $ " +"{:.1f}".format(np.log10(mcut))
                , fontsize=fontsize_ticks
                , transform=axs[2].transAxes)


    if examplegal is not None:
        dls = examplegal.merger.delta_l
        mrs = examplegal.merger.mr

        for dl, mr in zip(dls, mrs):
            if mr < maj_ratio:
                axs[0].scatter(dl, [0.15], facecolor='r', edgecolor="w", marker="d", s=40, zorder=20)
            else:
                axs[0].scatter(dl, [0.15], facecolor='g', edgecolor="w", marker="d", s=40, zorder=20)

        dl_tot = examplegal.data["lambda_r"][0] - examplegal.data["lambda_r"][-1]
        axs[0].scatter(dl_tot, [0.15], facecolor='black', edgecolor="w", marker="d", s=40, zorder=20)
        dl_o = dl_tot - sum(examplegal.merger.delta_l)
        axs[0].scatter(dl_o, 0.15, facecolor='b', edgecolor="w", marker="d", s=40, zorder=20)

        # legend
        axs[0].scatter(0.21, 1, facecolor='none', edgecolor="black", marker="d", s=40)
        axs[0].text(0.24, 0.95, "example galaxy", fontsize=8)
        print("Example gal")

    axs[0].set_ylim([0,3.5])
    axs[1].set_ylim([0,3.5])
    axs[2].set_ylim([0,3.5])


    plt.savefig(fname + "{:.1f}.png".format(np.log10(mcut)), dpi=150, bbox_inches="tight")
    plt.savefig(fname + "{:.1f}.pdf".format(np.log10(mcut)), bbox_inches='tight') # eps does NOT support transparency!
    #plt.savefig(fname + "{:.1f}.eps".format(np.log10(mcut)), bbox_inches='tight')
    #plt.savefig(fname + "{:.1f}.svg".format(np.log10(mcut)), bbox_inches='tight')

    #plt.close()
    return skew_fits

In [8]:
from glob import glob

In [9]:
fnames = glob("/home/hoseung/Work/data/all_prgs/main_prgs_final_augmented*02_filtered_.pickle")

In [12]:
fits=[]
for fname in fnames:
    mpgs = pickle.load(open(fname, "rb"))
    suffix = fname.split("augmented")[1].split("filtered")[0]
    fit = kde_sci(mpgs,
            mstar_cut_hard = 5e9,
            mcut = 3.3e10,
            hist=False,
            shade=False,
            kde=True,
            norm_hist=False,
            detected=True,
            maj_ratio=4,
            excess=False,
            fname=wdir + "figs/MajMinNon_contribution" + suffix,
            img_scale=1.5,
            do_fit=True)
    fits.append(fit)